In [19]:
## Creating Corpus
import string
import json


def load_data(file):
    with open(file,'r',encoding='utf-8') as f:
        data=f.read()
    return data

def save_data(file, data):
    
    
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

# full_book = './data/Harry_Potter_all_books_preprocessed.txt'
full_book = './data/hp.txt'

text=load_data(full_book)

In [20]:
# text = text.replace(" .",". ")

In [25]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


stop_words = set(stopwords.words("english"))
para=[]

for paras in text.split("\n\n"):
    word_token=word_tokenize(paras)
    clean_token=[token for token in word_token if token.casefold() not in stop_words and token.casefold() not in string.punctuation]
    para.append(clean_token)

save_data('./data/para.json',para)
print(len(para))

3030


In [33]:
import numpy as np
import spacy
import gensim
import multiprocessing

from gensim.models import Word2Vec 
from gensim.models import KeyedVectors


from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile


In [45]:
def training(model_name):
    with open('./data/para.json','r',encoding='utf-8') as f:
        texts=json.load(f)
    # print(texts)  
    
    sentences = texts
    
    # Getting the number of cores
    cores = multiprocessing.cpu_count()
    
    # Training the word2vec model
    w2v_model = Word2Vec(min_count=5, # Minimum word count threshold
                     window=2, # The maximum distance between the current and predicted word within a sentence
                     vector_size=500, # Embedding vector size
                     sample=6e-5, # Downsample setting for frequent words
                     alpha=0.03, # Learning rate
                     min_alpha=0.0007, # Minimum learning rate
                     negative=20, # If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown
                     workers=cores-1) # Number of threads to use
    
    # Building the Vocab
    w2v_model.build_vocab(sentences, progress_per=5000)
    
    # Training the model
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    
    # w2v_model.init_sims(replace=True)
    
    w2v_model.save(f"word_vectors/{model_name}.model")
    w2v_model.wv.save_word2vec_format(f"word_vectors/{model_name}.txt", binary=False)
    print("Training Complete")
    
    
training('hp_ner_01')

Training Complete


In [46]:
def gen_similarity(word):
    model = KeyedVectors. load_word2vec_format ('word_vectors/hp_ner_01.txt', binary=False)
    results = model.most_similar(positive=[word])
    return (results)

In [47]:
print(gen_similarity('Harry'))

[('Ron', 0.9997612237930298), ('yelled', 0.9997559785842896), ('noticed', 0.9997556805610657), ('quite', 0.9997545480728149), ('sat', 0.9997537732124329), ('bringing', 0.9997531771659851), ('stood', 0.9997504353523254), ('quickly', 0.9997499585151672), ('sprinted', 0.9997485280036926), ('leaned', 0.9997472763061523)]


In [ ]:

model = KeyedVectors. load_word2vec_format ('word_vectors/word2ve_hp_ner_model.txt', binary=False)

"""
SOURCE:
https://stackoverflow.com/questions/54717449/mapping-word-vector-to-the-most-similar-closest-word-using-spacy
"""

def spacy_similarity(your_word):
    nlp = spacy. load ("en_core_web_1g")
    
    ms = nlp. vocab.vectors.most_similar(
        np.asarray([nlp.vocab.vectors[nlp.vocab.strings[your_word]]]), n=10)
    
    words = [nlp.vocab.strings[w] for w in ms [0][0]]
    distances = ms[2]
    print (words)

spacy_similarity("albus")

In [ ]:
# ax.scatter(df['x'], df['y'])
# for word, pos in df.iterrows ():
# plt. annotate(word, pos)
# fig. savefig("hp_vectors")
# plt.show()

# # generate_plot()
# #
# # def analogy (x1, x2, y1):
# #
# result = model.most similarpositive=[y1, x2], negative=[x1])
# #
# #
# return result[0:10]
# #
# # result = analogy('Gryffindor', 'red', 'SLytherin')
# # print (result)
# #
# gen_similarity("Gryffindor")
# # display_pca_scatterplot(["Potter", "Ron"])